In [1]:
%load_ext autotime

In [2]:
import sys
sys.path.append('/Users/Pradap/Documents/Research/Python-Package/enrique/')

time: 883 µs


In [3]:
import magellan as mg
import pandas as pd

DEBUG:Cloud:Log file (/Users/pradap/.picloud/cloud.log) opened


time: 472 ms


In [4]:
mg.init_jvm('/Library/Java/JavaVirtualMachines/jdk1.8.0_45.jdk/Contents/Home/jre/lib/server/libjvm.dylib')

True

time: 95.3 ms


In [5]:
# read walmart dataset
wal =  mg.read_csv(mg.get_install_path() + '/datasets/books/walmart.csv', 
                    dtype={'isbn':pd.np.str, 'pages':pd.np.str, 'volume':pd.np.str, 'editionNum':pd.np.str},  
                    low_memory=False, key='id')

time: 1.17 s


In [6]:
bwk = mg.read_csv(mg.get_install_path() + '/datasets/books/bowker.csv', 
                  dtype={'isbn':pd.np.str, 'pages':pd.np.str, 'volume':pd.np.str, 'editionNum':pd.np.str},  
                  low_memory=False, key='id')

time: 84.5 ms


In [7]:
ab = mg.AttrEquivalenceBlocker()
C = ab.block_tables(wal, bwk, 'isbn', 'isbn', ['title', 'author'], 
                   ['title', 'author'])

time: 192 ms


In [8]:
feat_table = mg.get_features_for_blocking(wal, bwk)

time: 3.22 s


In [9]:
rb = mg.RuleBasedBlocker()
rb.add_rule('title_title_jac_qgm_3_qgm_3(ltuple, ltuple) < 0.8', feat_table)

True

time: 2.58 ms


In [10]:
D = rb.block_candset(C)

time: 675 ms
